## Notebook to determine important carriers for the STST and EXP networks

In [1]:
# imports
import pypsa

In [2]:
# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [7]:
print(stst)

PyPSA Network
Components:
 - Bus: 3086
 - Carrier: 30
 - Generator: 1707
 - GlobalConstraint: 3
 - Line: 323
 - Link: 8375
 - Load: 2542
 - StorageUnit: 170
 - Store: 1635
Snapshots: 2920


In [8]:
print(exp)

PyPSA Network
Components:
 - Bus: 3086
 - Carrier: 30
 - Generator: 1707
 - GlobalConstraint: 2
 - Line: 323
 - Link: 9192
 - Load: 2542
 - StorageUnit: 170
 - Store: 1635
Snapshots: 2920


### Carrier differences

In [11]:
# same carriers for generators
set(exp.generators.carrier.unique().tolist()).difference(stst.generators.carrier.unique().tolist())
stst.generators.carrier.unique().tolist()

['offwind-ac',
 'onwind',
 'solar',
 'ror',
 'offwind-dc',
 'gas',
 'residential rural solar thermal',
 'services rural solar thermal',
 'residential urban decentral solar thermal',
 'services urban decentral solar thermal',
 'urban central solar thermal',
 'oil',
 'solar rooftop']

In [15]:
# EXP has two additional links: 'H2 pipeline', 'H2 pipeline retrofitted'
set(exp.links.carrier.unique().tolist()).difference(stst.links.carrier.unique().tolist())
exp.links.carrier.unique().tolist()

['DC',
 'OCGT',
 'H2 Electrolysis',
 'H2 Fuel Cell',
 'H2 pipeline retrofitted',
 'H2 pipeline',
 'battery charger',
 'battery discharger',
 'Sabatier',
 'SMR CC',
 'SMR',
 'BEV charger',
 'V2G',
 'residential rural ground heat pump',
 'residential rural water tanks charger',
 'residential rural water tanks discharger',
 'residential rural resistive heater',
 'residential rural gas boiler',
 'services rural ground heat pump',
 'services rural water tanks charger',
 'services rural water tanks discharger',
 'services rural resistive heater',
 'services rural gas boiler',
 'residential urban decentral air heat pump',
 'residential urban decentral water tanks charger',
 'residential urban decentral water tanks discharger',
 'residential urban decentral resistive heater',
 'residential urban decentral gas boiler',
 'services urban decentral air heat pump',
 'services urban decentral water tanks charger',
 'services urban decentral water tanks discharger',
 'services urban decentral resisti

In [18]:
# same carriers for storage units
set(stst.storage_units.carrier.unique().tolist()).difference(exp.storage_units.carrier.unique().tolist())
stst.storage_units.carrier.unique().tolist()

['hydro', 'PHS']

### Relevant carriers per technology
- classify in electricity (gen / con), hydrogen (gen, con) and heat (gen, con)
- determine by share of generation / consumption

In [ ]:
# investigate in carriers with most generation for both networks
# without grid ("electricity distribution grid", "DC", "H2 pipeline", "H2 pipeline retrofitted")
# storage technologies:

carriers_el_gen = [
    "electricity distribution grid",
    "onwind",
    "solar",
    "BEV charger",
    "offwind-dc",
    "ror",
    "hydro",
    "solar rooftop",
    "offwind-ac",
    "battery charger",
    "battery discharger",
    "urban central solid biomass CHP CC",
    "V2G",
    "PHS",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "OCGT",
    "home battery charger",
    "home battery discharger",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DC"
]

carriers_el_con = [
    "PHS",
    "electricity distribution grid",
    "H2 Electrolysis",
    "BEV charger",
    "urban central air heat pump",
    "residential urban decentral air heat pump",
    "residential rural ground heat pump",
    "services rural ground heat pump",
    "services urban decentral air heat pump",
    "urban central resistive heater",
    "battery charger",
    "battery discharger",
    "V2G",
    "residential urban decentral resistive heater",
    "residential rural resistive heater",
    "services urban decentral resistive heater",
    "services rural resistive heater",
    "home battery charger",
    "home battery discharger",
    "DC",
    "DAC",
]

carriers_h2_gen = [
    "H2 Electrolysis",
    "SMR CC",
    "SMR"
]

carriers_h2_con = [
    "H2 liquefaction",
    "H2 Fuel Cell",
    "Sabatier",
    "Fischer-Tropsch"
]

carriers_heat_gen1 = [
    "urban central air heat pump",
    "residential urban decentral air heat pump",
    "residential rural ground heat pump",
    "services rural ground heat pump",
    "services urban decentral air heat pump",
    "urban central gas boiler",
    "urban central resistive heater",
    "residential urban decentral gas boiler",
    "urban central water tanks charger",
    "urban central water tanks discharger",
    "residential urban decentral resistive heater",
    "services urban decentral gas boiler",
    "urban central solar thermal",
    "residential rural resistive heater",
    "services urban decentral resistive heater",
    "residential rural gas boiler",
    "services rural resistive heater",
    "services rural gas boiler",
    "services rural water tanks charger",
    "residential rural water tanks charger",
    "services rural water tanks discharger",
    "residential rural water tanks discharger",
    "services urban decentral water tanks charger",
    "residential urban decentral water tanks charger",
    "services urban decentral water tanks discharger",
    "residential urban decentral water tanks discharger",
    "residential urban decentral solar thermal",
    "services urban decentral solar thermal",
    "residential rural solar thermal",
    "services rural solar thermal"
]

carriers_heat_gen2 = [
    "urban central solid biomass CHP CC",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DAC"
]

carriers_heat_con = [
"urban central water tanks charger",
"urban central water tanks discharger",
"services rural water tanks charger",
"residential rural water tanks charger",
"services rural water tanks discharger",
"residential rural water tanks discharger",
"services urban decentral water tanks charger",
"residential urban decentral water tanks charger",
"services urban decentral water tanks discharger",
"residential urban decentral water tanks discharger",
]

# for gas, biomass, oil, process emission, C02 refer to excel table